In [1]:
import io
import base64
import pandas as pd
import numpy as np
from helper_fcts import preprocess, get_poi_mas, rps
from keras import backend as K
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.preprocessing import scale, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, KFold, cross_val_score
from sklearn.svm import SVR, SVC
from keras.models import Sequential
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
from xgboost import XGBClassifier, XGBRegressor
import xgboost as xgb
import matplotlib.pyplot as plt
from scipy.interpolate import spline
from IPython.display import SVG, HTML
from operator import itemgetter
from scipy.stats import poisson
from mpl_toolkits.mplot3d import Axes3D
from prediction import get_predictions
from keras import regularizers
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib

from main import modelling

C:\Users\Konny\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
eng1 = pd.concat([modelling(pd.read_csv("data/E" + str(i) + ".csv"), 20) for i in range(8,19)])
eng2 = pd.concat([modelling(pd.read_csv("data/E1_" + str(i) + ".csv"), 24) for i in range(8,19)])
eng3 = pd.concat([modelling(pd.read_csv("data/E2_" + str(i) + ".csv"), 24) for i in range(8,19)])
eng4 = pd.concat([modelling(pd.read_csv("data/E3_" + str(i) + ".csv"), 24) for i in range(8,19)])
fra = pd.concat([modelling(pd.read_csv("data/F" + str(i) + ".csv"), 20) for i in range(8,19)])
ger = pd.concat([modelling(pd.read_csv("data/D" + str(i) + ".csv"), 18) for i in range(8,19)])
ita = pd.concat([modelling(pd.read_csv("data/I" + str(i) + ".csv"), 20) for i in range(8,19)])
spa = pd.concat([modelling(pd.read_csv("data/SP" + str(i) + ".csv"), 20) for i in range(8,19)])

C:\Users\Konny\Anaconda3\lib\site-packages\scipy\stats\_discrete_distns.py:493: RuntimeWarning: invalid value encountered in greater_equal
  return mu >= 0


In [4]:
fra2 = modelling(pd.read_csv("F2.csv"), 20)
ger2 = modelling(pd.read_csv("D2.csv"), 18)
ita2 = modelling(pd.read_csv("I2.csv"), 22)
spa2 = modelling(pd.read_csv("SP2.csv"), 22)
bel = modelling(pd.read_csv("B1.csv"), 16)
gre = modelling(pd.read_csv("G1.csv"), 16)
ned = modelling(pd.read_csv("N1.csv"), 18)
por = modelling(pd.read_csv("P1.csv"), 18)
tur = modelling(pd.read_csv("T1.csv"), 18)

C:\Users\Konny\Anaconda3\lib\site-packages\scipy\stats\_discrete_distns.py:493: RuntimeWarning: invalid value encountered in greater_equal
  return mu >= 0


In [4]:
eng1["Country"] = "E1"
eng2["Country"] = "E2"
eng3["Country"] = "E3"
eng4["Country"] = "E4"
fra["Country"] = "F"
ger["Country"] = "D"
ita["Country"] = "I"
spa["Country"] = "S"

In [5]:
df = pd.concat([eng1, eng2, eng3, eng4, fra, ger, ita, spa, fra2, ger2, ita2, spa2, bel, gre, ned, por, tur]).reset_index(drop=True)

In [6]:
df = df[df["round"] > 10]

In [7]:
df.reset_index(drop=True, inplace=True)

In [15]:
dummies = pd.get_dummies(df["Country"])

In [17]:
df = pd.concat([df, dummies], axis=1)

In [8]:
columns_mit_fe  = ['FTR', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_avgC', 'A_avgC', 'H_avgC_c', 'A_avgC_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]
df = df.iloc[df[columns_mit_fe].dropna().index]

In [10]:
df.to_pickle("data_other_countries.pkl")

In [2]:
df = pd.read_pickle("data_all.pkl")

In [11]:
year = 18

In [4]:
mapper = {
    "E1": "E0",
    "E2": "E1",
    "E3": "E2",
    "E4": "E3",
    "F": "F1",
    "D": "D1",
    "I": "I1",
    "S": "SP1"
}

In [5]:
df = df.rename(mapper, axis="columns")

## 6. Predicting Goal Difference instead of Classification

#### XGB Regressor

In [71]:
#columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_avgC', 'A_avgC', 'H_avgC_c', 'A_avgC_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds", "D1", "E0", "E1", "E2", "E3", "F1", "I1", "SP1"]
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_avgC', 'A_avgC', 'H_avgC_c', 'A_avgC_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]


def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,2:]
    X_test = df[df["season"] == season].iloc[:,2:]
    y_train_hg = df[df["season"] < season].iloc[:,0]
    y_test_hg = df[df["season"] == season].iloc[:,0]
    y_train_ag = df[df["season"] < season].iloc[:,1]
    y_test_ag = df[df["season"] == season].iloc[:,1]
    return X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag

data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

# X_train = data.iloc[:,2:]
# y_train_hg = data.iloc[:,0]
# y_train_ag = data.iloc[:,1]

X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag = get_season(data, year)

del X_train["season"]
del X_test["season"]

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [148]:
data2 = ned[ned["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_test = data.iloc[:,2:]
del X_test["season"]
DMtest = xgb.DMatrix(data=X_test)
X_test_scaled = scaler.transform(X_test)

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [149]:
xgb_home = xg_reg_h3.predict(DMtest)
xgb_away = xg_reg_a3.predict(DMtest)

In [150]:
lin_home = model_h.predict(X_test)
lin_away = model_a.predict(X_test)

In [151]:
y_pred = model.predict(X_test_scaled)
y_pred = scaler_goals.inverse_transform(y_pred)

In [72]:
#data
DMtrain_h = xgb.DMatrix(data=X_train, label = y_train_hg)
DMtest_h = xgb.DMatrix(data=X_test)
DMtrain_a = xgb.DMatrix(data=X_train, label = y_train_ag)
DMtest_a = xgb.DMatrix(data=X_test)

In [73]:
params = {"booster":"gblinear", "objective":"reg:linear", "max_depth":4, 'min_child_weight': 5, "n_estimators": 100, 'eval_metric': "rmse", 'seed':11}

#home
xg_reg_h3 = xgb.train(params=params, dtrain=DMtrain_h,num_boost_round=100)
xgb_home = xg_reg_h3.predict(DMtest_h)

# #away
xg_reg_a3 = xgb.train(params=params, dtrain=DMtrain_a,num_boost_round=100)
xgb_away = xg_reg_a3.predict(DMtest_a)

In [74]:
# joblib.dump(xg_reg_h3, "other_xgb_h.model")
# joblib.dump(xg_reg_a3, "other_xgb_a.model")

['other_xgb_a.model']

In [15]:
test_df = data2.iloc[data[data["season"] == year].index]
test_df.reset_index(drop=True, inplace=True)
test_df["H_xgb_xG"] = xgb_home
test_df["A_xgb_xG"] = xgb_away
test_df["XGB_hc"] = test_df["A_xgb_xG"] - test_df["H_xgb_xG"]
test_df["Hc_Diff"] = test_df["BbAHh"] - test_df["XGB_hc"]

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [16]:
test_df[(test_df["Hc_Diff"] > 0.2) & (test_df["BbAvAHH"] > 1.8)].H_Ahc_PL.describe()

count    572.000000
mean       0.050979
std        0.861097
min       -1.000000
25%       -1.000000
50%        0.435000
75%        0.870000
max        1.170000
Name: H_Ahc_PL, dtype: float64

In [17]:
test_df[(test_df["Hc_Diff"] < -0.2) & (test_df["BbAvAHA"] > 1.8)].A_Ahc_PL.describe()

count    318.000000
mean       0.084230
std        0.861031
min       -1.000000
25%       -1.000000
50%        0.437500
75%        0.880000
max        1.100000
Name: A_Ahc_PL, dtype: float64

### Lineare Regression

In [18]:
#columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_avgC', 'A_avgC', 'H_avgC_c', 'A_avgC_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds", "D1", "E0", "E1", "E2", "E3", "F1", "I1", "SP1"]
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_avgC', 'A_avgC', 'H_avgC_c', 'A_avgC_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]

def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,2:]
    X_test = df[df["season"] == season].iloc[:,2:]
    y_train_hg = df[df["season"] < season].iloc[:,0]
    y_test_hg = df[df["season"] == season].iloc[:,0]
    y_train_ag = df[df["season"] < season].iloc[:,1]
    y_test_ag = df[df["season"] == season].iloc[:,1]
    return X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag

data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_train, X_test, y_train_hg, y_test_hg, y_train_ag, y_test_ag = get_season(data, year)


del X_train["season"]
del X_test["season"]

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
#home
model_h = LinearRegression()
model_h.fit(X_train, y_train_hg)
lin_home = model_h.predict(X_test)

#away
model_a = LinearRegression()
model_a.fit(X_train, y_train_ag)
lin_away = model_a.predict(X_test)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [76]:
# joblib.dump(model_h, 'other_lin_reg_h.joblib')
# joblib.dump(model_a, 'other_lin_reg_a.joblib')

['other_lin_reg_a.joblib']

In [20]:
test_df = data2.iloc[data[data["season"] == year].index]
test_df.reset_index(drop=True, inplace=True)
test_df["H_lin_xG"] = lin_home
test_df["A_lin_xG"] = lin_away
test_df["LIN_hc"] = test_df["A_lin_xG"] - test_df["H_lin_xG"]
test_df["Hc_Diff"] = test_df["BbAHh"] - test_df["LIN_hc"]

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/st

In [21]:
test_df[(test_df["Hc_Diff"] > 0.2) & (test_df["BbAvAHH"] > 1.8)].H_Ahc_PL.describe()

count    637.000000
mean       0.050345
std        0.864791
min       -1.000000
25%       -1.000000
50%        0.435000
75%        0.870000
max        1.170000
Name: H_Ahc_PL, dtype: float64

In [22]:
test_df[(test_df["Hc_Diff"] < -0.2) & (test_df["BbAvAHA"] > 1.8)].A_Ahc_PL.describe()

count    296.000000
mean       0.095625
std        0.852753
min       -1.000000
25%       -1.000000
50%        0.437500
75%        0.880000
max        1.100000
Name: A_Ahc_PL, dtype: float64

### Artificial Neural Network

In [77]:
#columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_avgC', 'A_avgC', 'H_avgC_c', 'A_avgC_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds", "D1", "E0", "E1", "E2", "E3", "F1", "I1", "SP1"]
columns_mit_fe  = ['FTHG', 'FTAG', 'season', 'H_avgGD', 'A_avgGD', 'H_avgG', 'A_avgG', 'H_avgG_c', 'A_avgG_c', 'H_avgST', 'A_avgST', 'H_avgST_c', 'A_avgST_c', 'H_avgC', 'A_avgC', 'H_avgC_c', 'A_avgC_c', 'H_GoalDiff_last', 'A_GoalDiff_last', 'H_xG_PoiMas', 'A_xG_PoiMas', 'H_Form_Tot4', 'A_Form_Tot4','H_Def_Rat', 'H_Off_Rat', 'A_Def_Rat', 'A_Off_Rat', "H_prob_odds", "D_prob_odds", "A_prob_odds"]


def get_season(df, season):
    X_train = df[df["season"] < season].iloc[:,2:]
    X_test = df[df["season"] == season].iloc[:,2:]
    y_train = df[df["season"] < season].iloc[:,:2]
    y_test = df[df["season"] == season].iloc[:,:2]
    return X_train, X_test, y_train, y_test


data2 = df[df["round"] > 10].reset_index(drop=True)
data = data2[columns_mit_fe]
data.dropna(inplace=True)

X_train, X_test, y_train, y_test = get_season(data, year)

del X_train["season"]
del X_test["season"]

scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [24]:
scaler_goals = StandardScaler()
scaler_goals.fit(y_train)

y_train = scaler_goals.transform(y_train)
y_test = scaler_goals.transform(y_test)

In [79]:
K.clear_session()

model = Sequential()
model.add(Dense(128, kernel_initializer="he_normal", input_shape=(27,)))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(64, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(32, kernel_initializer="he_normal"))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(2, kernel_initializer="he_normal"))
model.add(Activation('linear'))

model.compile(loss="logcosh", optimizer='adam', metrics=['accuracy'])

In [80]:
history = model.fit(X_train, y_train, epochs=100)

Epoch 1/100
31000/31000 [==============================] - 5s 160us/step - loss: 0.5069 - acc: 0.5494
Epoch 2/100
31000/31000 [==============================] - 4s 114us/step - loss: 0.3512 - acc: 0.6002
Epoch 3/100
31000/31000 [==============================] - 4s 120us/step - loss: 0.3462 - acc: 0.6124
Epoch 4/100
31000/31000 [==============================] - 3s 113us/step - loss: 0.3449 - acc: 0.6117
Epoch 5/100
31000/31000 [==============================] - 4s 122us/step - loss: 0.3442 - acc: 0.6134
Epoch 6/100
31000/31000 [==============================] - 4s 132us/step - loss: 0.3439 - acc: 0.6153
Epoch 7/100
31000/31000 [==============================] - 4s 122us/step - loss: 0.3437 - acc: 0.6118
Epoch 8/100
31000/31000 [==============================] - 4s 114us/step - loss: 0.3433 - acc: 0.6135
Epoch 9/100
31000/31000 [==============================] - 3s 112us/step - loss: 0.3431 - acc: 0.6132
Epoch 10/100
31000/31000 [==============================] - 3s 112us/step - loss: 

31000/31000 [==============================] - 3s 88us/step - loss: 0.3398 - acc: 0.6146
Epoch 72/100
31000/31000 [==============================] - 3s 85us/step - loss: 0.3398 - acc: 0.6192
Epoch 73/100
31000/31000 [==============================] - 3s 83us/step - loss: 0.3400 - acc: 0.6215
Epoch 74/100
31000/31000 [==============================] - 3s 95us/step - loss: 0.3400 - acc: 0.6202
Epoch 75/100
31000/31000 [==============================] - 3s 87us/step - loss: 0.3397 - acc: 0.6189
Epoch 76/100
31000/31000 [==============================] - 3s 88us/step - loss: 0.3402 - acc: 0.6239
Epoch 77/100
31000/31000 [==============================] - 3s 96us/step - loss: 0.3401 - acc: 0.6202
Epoch 78/100
31000/31000 [==============================] - 3s 89us/step - loss: 0.3396 - acc: 0.6206
Epoch 79/100
31000/31000 [==============================] - 3s 85us/step - loss: 0.3399 - acc: 0.6177
Epoch 80/100
31000/31000 [==============================] - 3s 93us/step - loss: 0.3400 - acc: 

In [81]:
print(f"Train Dataset:\n RPS: {np.round(model.evaluate(X_train, y_train, verbose=0)[0],4)}, Accuracy: {np.round(model.evaluate(X_train, y_train, verbose=0)[1],4)}")
print("--------------------------------------------")
#print(f"Test Dataset:\n RPS: {np.round(model.evaluate(X_test, y_test, verbose=0)[0],4)}, Accuracy: {np.round(model.evaluate(X_test, y_test, verbose=0)[1],4)}")
print("--------------------------------------------")

Train Dataset:
 RPS: 0.3324, Accuracy: 0.6299
--------------------------------------------
--------------------------------------------


In [82]:
# model.save('other_ann_reg.h5')

In [22]:
plt.style.use('ggplot')

def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()

In [23]:
plot_history(history)

KeyError: 'val_acc'

In [29]:
y_pred = model.predict(X_test)

y_pred = scaler_goals.inverse_transform(y_pred)

test_df = data2.iloc[data[data["season"] == year].index]
test_df.reset_index(drop=True, inplace=True)

test_df["H_xgb_xG"] = xgb_home
test_df["A_xgb_xG"] = xgb_away
test_df["XGB_hc"] = test_df["A_xgb_xG"] - test_df["H_xgb_xG"]
test_df["Hc_Diff_xgb"] = test_df["BbAHh"] - test_df["XGB_hc"]

test_df["H_lin_xG"] = lin_home
test_df["A_lin_xG"] = lin_away
test_df["LIN_hc"] = test_df["A_lin_xG"] - test_df["H_lin_xG"]
test_df["Hc_Diff_lin"] = test_df["BbAHh"] - test_df["LIN_hc"]

ann_pred2 = pd.DataFrame(y_pred, columns = ["H_ann_xG", "A_ann_xG"])
test_df2 = pd.concat([test_df, ann_pred2], axis=1)
test_df2["ANN_hc"] = test_df2["A_ann_xG"] - test_df2["H_ann_xG"]
test_df2["Hc_Diff_ann"] = test_df2["BbAHh"] - test_df2["ANN_hc"]

C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Konny\Anaconda3\lib\site-packages\ipykernel\__main__.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing

In [30]:
difference = 0.2

test_df2[(test_df2["Hc_Diff_ann"] > difference) & (test_df2["BbAvAHH"] > 1.8)].H_Ahc_PL.describe()

count    736.000000
mean       0.048743
std        0.846491
min       -1.000000
25%       -1.000000
50%        0.000000
75%        0.880000
max        1.200000
Name: H_Ahc_PL, dtype: float64

In [31]:
test_df2[(test_df2["Hc_Diff_ann"] < -difference) & (test_df2["BbAvAHA"] > 1.8)].A_Ahc_PL.describe()

count    321.000000
mean       0.094346
std        0.841474
min       -1.000000
25%       -1.000000
50%        0.425000
75%        0.880000
max        1.830000
Name: A_Ahc_PL, dtype: float64

In [32]:
test_df2["Hc_Diff_avg"] = (test_df2["Hc_Diff_ann"] + test_df2["Hc_Diff_xgb"] + test_df2["Hc_Diff_lin"]) / 3

In [33]:
test_df2[["HomeTeam", "AwayTeam", "Hc_Diff_ann", "Hc_Diff_xgb", "Hc_Diff_lin", "Hc_Diff_avg"]].tail()

,HomeTeam,AwayTeam,Hc_Diff_ann,Hc_Diff_xgb,Hc_Diff_lin,Hc_Diff_avg
4702,Genclerbirligi,Bursaspor,0.329820,0.400807,0.494551,0.408392
4703,Yeni Malatyaspor,Kayserispor,0.144297,0.115148,0.125183,0.128210
4704,Besiktas,Sivasspor,-0.368040,-0.328886,-0.310334,-0.335754
4705,Buyuksehyr,Kasimpasa,-0.537176,-0.190899,-0.133414,-0.287163
4706,Goztep,Galatasaray,0.633845,0.763114,0.689903,0.695621


In [69]:
difference = 0.15

test_df2[(test_df2["Hc_Diff_avg"] > difference) & (test_df2["BbAvAHH"] > 1.8) & (test_df2["Div"].isin(["B1", "D2", "F2", "G1", "I2", "N1", "P1", "T1", "SP2"]))].groupby("Div").H_Ahc_PL.describe()

,count,mean,std,min,25%,50%,75%,max
Div,,,,,,,,
B1,32.0,-0.367188,0.717515,-1.0,-1.0,-0.5000,0.105,0.93
D2,36.0,0.071806,0.820220,-1.0,-1.0,0.4225,0.825,1.01
F2,41.0,0.164512,0.875214,-1.0,-1.0,0.8100,0.870,1.11
G1,22.0,-0.036364,0.858601,-1.0,-1.0,-0.2500,0.880,1.03
I2,57.0,0.101842,0.832034,-1.0,-1.0,0.4300,0.840,1.04
N1,43.0,-0.028605,0.904240,-1.0,-1.0,0.0000,0.865,1.07
P1,28.0,0.118929,0.900783,-1.0,-1.0,0.4100,0.925,1.09
SP2,79.0,0.111203,0.861193,-1.0,-1.0,0.4550,0.880,1.20
T1,43.0,0.035000,0.852772,-1.0,-1.0,0.0000,0.880,1.01


In [68]:
test_df2[(test_df2["Hc_Diff_avg"] < -difference) & (test_df2["BbAvAHA"] > 1.8) & (test_df2["Div"].isin(["B1", "D2", "F2", "G1", "I2", "N1", "P1", "T1", "SP2"]))].groupby("Div").A_Ahc_PL.describe()

,count,mean,std,min,25%,50%,75%,max
Div,,,,,,,,
B1,20.0,0.123500,0.890826,-1.0,-1.00,0.630,0.8850,0.99
D2,11.0,0.233636,0.890935,-1.0,-0.75,0.830,0.8550,1.00
F2,17.0,0.102059,0.812952,-1.0,-0.50,0.405,0.8600,0.89
G1,11.0,0.048636,0.756023,-1.0,-0.50,0.000,0.6525,0.98
I2,17.0,-0.045000,0.813772,-1.0,-1.00,0.000,0.8200,0.98
N1,22.0,-0.101818,0.916030,-1.0,-1.00,-0.250,0.9100,0.99
P1,23.0,0.247609,0.735797,-1.0,-0.25,0.445,0.8450,1.01
SP2,16.0,-0.350000,0.813691,-1.0,-1.00,-1.000,0.2050,0.87
T1,28.0,-0.015000,0.879716,-1.0,-1.00,0.000,0.8725,0.96


In [42]:
difference = 0.1

test_df2[(test_df2["Hc_Diff_ann"] > difference) & (test_df2["Hc_Diff_lin"] > difference) & (test_df2["Hc_Diff_xgb"] > difference) & (test_df2["BbAvAHH"] > 1.8)].H_Ahc_PL.describe()

count    645.000000
mean       0.081488
std        0.858445
min       -1.000000
25%       -1.000000
50%        0.440000
75%        0.880000
max        1.200000
Name: H_Ahc_PL, dtype: float64

In [43]:
difference = -0.1

test_df2[(test_df2["Hc_Diff_ann"] < difference) & (test_df2["Hc_Diff_lin"] < difference) & (test_df2["Hc_Diff_xgb"] < difference) & (test_df2["BbAvAHA"] > 1.8)].A_Ahc_PL.describe()

count    234.000000
mean       0.085876
std        0.854228
min       -1.000000
25%       -1.000000
50%        0.430000
75%        0.880000
max        1.090000
Name: A_Ahc_PL, dtype: float64